# Simulation template 

In this notebook we run the machine learning analysis of topological phase transitions occurring  in both nearest-neighbours SSH models (ssh1) and second neighbours models (ssh2) as decribed in the paper [Machine learning topological phases in real space](https://arxiv.org/abs/1901.01963). Here the simulation is run with features generated from fourier components in the first scenario. This scenario is characterized by using only the EVEN wavevector space eigenmodes, computed from ALL real space components.

## Defining parameters

In [1]:
%%time
%load_ext autoreload
%autoreload 2
from simulation import *

CPU times: user 754 ms, sys: 267 ms, total: 1.02 s
Wall time: 584 ms


In [2]:
%%time
### Dataset and simulation parameters
#csv_path = SSH2_PERIODIC_100_6561_CSV 
csv_path = SSH1_PERIODIC_100_6561_CSV 
model_name = "DecisionTreeClassifier"
model_kw = {"criterion":"entropy"}
allowed_windings = [0,1]
#allowed_windings = [-1,0,1,2]
simulation_dir = SSH1_REAL_SPACE_ALL_SITES
#simulation_dir = SSH2_PERIODIC_ENGINEERED_100_6561_SIMULATION_DIR
val_split = 0.9  
features_to_use = None
shuffle_features = False
random_state = 763

### Fourier features
fourier_mode = "dst"
fourier_features_to_use = None
fourier_real = "imag"
fourier_normalize = False
fourier_fillna = False

### Running a simulation
n_experiments = 5
start_n = 0
fit_params = None
#mode=mode
shuffle_rows = True
pred_params = None
random_features = False
######### DON'T SET THIS TO TRUE UNLESS YOUR DATASET IS SMALL!! WILL FLOOD YOUR MEMORY!!!
store_in_lists = False   
########## BELOW ARE THE PARAMETERS THAT CONTROL WHAT WILL BE SAVED
save_eigenvector=True
save_hamiltonian=True 
save_accuracy=True 
save_models=True

CPU times: user 3 µs, sys: 0 ns, total: 3 µs
Wall time: 4.53 µs


In [3]:
# Parameters
model_kw = {"criterion": "entropy"}
allowed_windings = [0, 1]
val_split = 0.9
features_to_use = None
shuffle_features = False
n_experiments = 100
start_n = 0
fit_params = None
shuffle_rows = True
pred_params = None
random_features = False
store_in_lists = False
save_eigenvector = True
save_hamiltonian = True
save_accuracy = True
save_models = True
csv_path = "/home/rio/ssh_csvs/ssh1/periodic_100_6561.csv"
model_name = "DecisionTreeClassifier"
simulation_dir = (
    "/home/rio/ssh_simulations/ssh1/periodic_100_6561/dst_best_04_wavevector_sites"
)
random_state = 1019
fourier_features_to_use = [0, 18, 28, 30]
fourier_mode = "dst"
fourier_real = None
fourier_normalize = False
fourier_fillna = False


In [4]:
%%time
#Starting an instance of Simulation with a decision tree
model = DecisionTreeClassifier(criterion="entropy")
#dict_args = {"csv_path":csv_path, "model":model, "allowed_windings":allowed_windings,\
#             "simulation_dir":simulation_dir, "val_split":val_split, "features_to_use":features_to_use,\
#            "shuffle_features":shuffle_features, "random_state":random_state}
simulation = Simulation(csv_path,model_name,model_kw,allowed_windings,simulation_dir,val_split,features_to_use,\
                       shuffle_features,fourier_mode,fourier_features_to_use,fourier_real,\
                       fourier_normalize, fourier_fillna , random_state)
#simulation = Simulation(**dict_args)


#(self, csv_path, model_name, model_kw, allowed_windings, simulation_dir = None, val_split = 0, 
# features_to_use = None, shuffle_features = False, fourier_mode=None, fourier_features_to_use=None,
# fourier_real = False, fourier_normalize = False, Fourier_fillna=False, random_state = None):



print("Info on all data: \n")
simulation.dataframe.info()
simulation.dataframe.head()

Info on all data: 

<class 'pandas.core.frame.DataFrame'>
Int64Index: 656100 entries, 0 to 656099
Columns: 108 entries, id to feat99
dtypes: float64(103), int32(3), object(2)
memory usage: 538.1+ MB
CPU times: user 5.91 s, sys: 600 ms, total: 6.51 s
Wall time: 5.9 s


,id,path,t1,t2,winding,phase,pred_phase,type_of,feat0,feat1,...,feat90,feat91,feat92,feat93,feat94,feat95,feat96,feat97,feat98,feat99
0,0,H_-2_-2_0.509296.dat,-2.0,-2.0,0.509296,999,666,test,0.100000,0.100000,...,0.100000,0.100000,0.100000,0.100000,0.100000,0.100000,0.100000,0.100000,0.100000,0.100000
1,0,H_-2_-2_0.509296.dat,-2.0,-2.0,0.509296,999,666,test,0.141421,0.141142,...,0.114412,0.108967,0.123928,0.119406,0.131490,0.127962,0.136978,0.134500,0.140306,0.138916
2,0,H_-2_-2_0.509296.dat,-2.0,-2.0,0.509296,999,666,test,0.141421,0.140306,...,0.043702,0.026500,0.075777,0.060214,0.103092,0.090145,0.123928,0.114412,0.136978,0.131490
3,0,H_-2_-2_0.509296.dat,-2.0,-2.0,0.509296,999,666,test,-0.100000,0.100000,...,-0.100000,0.100000,-0.100000,0.100000,-0.100000,0.100000,-0.100000,0.100000,-0.100000,0.100000
4,0,H_-2_-2_0.509296.dat,-2.0,-2.0,0.509296,999,666,test,0.000509,-0.008372,...,-0.082713,-0.089753,-0.067684,-0.075347,-0.051587,-0.059753,-0.034677,-0.043217,-0.017220,-0.026000


####  Fourier features

In [5]:
%%time
#simulation.generate_fourier_df()
if simulation.fourier_mode is not None:
    simulation.fourier_dataframe.info() 
    simulation.fourier_dataframe.sample(15)


<class 'pandas.core.frame.DataFrame'>
Int64Index: 656100 entries, 0 to 656099
Data columns (total 12 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   id          656100 non-null  int32  
 1   path        656100 non-null  object 
 2   t1          656100 non-null  float64
 3   t2          656100 non-null  float64
 4   winding     656100 non-null  float64
 5   phase       656100 non-null  int32  
 6   pred_phase  656100 non-null  int32  
 7   type_of     656100 non-null  object 
 8   dst_feat0   656100 non-null  float64
 9   dst_feat18  656100 non-null  float64
 10  dst_feat28  656100 non-null  float64
 11  dst_feat30  656100 non-null  float64
dtypes: float64(7), int32(3), object(2)
memory usage: 57.6+ MB
CPU times: user 75.7 ms, sys: 0 ns, total: 75.7 ms
Wall time: 75 ms


In [6]:
print("features_to_use: ")
print("\n")
print(simulation.features_to_use)
if simulation.features_to_use is not None:
    print("Number of real space features:")
    print("\n")
    print(len(simulation.features_to_use))
print("\n")
print("fourier_features: ")
print("\n")
print(simulation.fourier_features)
print("\n")
if simulation.fourier_features is not None:
    print("Number of Fourier features: ")
    print("\n")
    print(len(simulation.fourier_features))
print("\n")
print("Fourier operator: ")
print(simulation.fourier_operator)
print("\n")
if simulation.fourier_operator is not None:
    print("Shape of Fourier operator: ")
    print(simulation.fourier_operator.shape)

features_to_use: 


None


fourier_features: 


['dst_feat0', 'dst_feat18', 'dst_feat28', 'dst_feat30']


Number of Fourier features: 


4


Fourier operator: 
[[ 0.12558104  1.85955297  1.93716632  1.85955297]
 [ 0.25066647  1.36909421 -0.96350735 -1.36909421]
 [ 0.37476263 -0.85155858 -1.45793725 -0.85155858]
 [ 0.49737977 -1.99605346  1.68865585  1.99605346]
 [ 0.61803399 -0.61803399  0.61803399 -0.61803399]
 [ 0.73624911  1.54102649 -1.99605346 -1.54102649]
 [ 0.85155858  1.75261336  0.37476263  1.75261336]
 [ 0.96350735 -0.25066647  1.8096541   0.25066647]
 [ 1.07165359 -1.93716632 -1.27484798 -1.93716632]
 [ 1.1755705  -1.1755705  -1.1755705   1.1755705 ]
 [ 1.27484798  1.07165359  1.85955297  1.07165359]
 [ 1.36909421  1.9645745   0.25066647 -1.9645745 ]
 [ 1.45793725  0.37476263 -1.9842294   0.37476263]
 [ 1.54102649 -1.68865585  0.73624911  1.68865585]
 [ 1.61803399 -1.61803399  1.61803399 -1.61803399]
 [ 1.68865585  0.49737977 -1.54102649 -0.49737977]
 [ 1.75261336  1.9842294

In [7]:
#a = np.array([1+1j,2+3j,4+8j])
#np.imag(a)

In [8]:
#plt.plot(np.imag(simulation.fourier_operator[:,0]))

In [9]:
#np.sin(np.pi/(50))*2

In [10]:
#2*np.sin(np.pi/50*49)

In [11]:
#simulation.dataframe.iloc[331456,:]

In [12]:
#for ix, obj in enumerate(simulation.dataframe.iloc[331456,:].values):
#    print(ix,obj)

In [13]:
#simulation.fourier_engineered_dataframe.dct_feat0

In [14]:
#simulation.features

In [15]:
#simulation.fourier_matrix

In [16]:
#1/simulation.n_features*simulation.dataframe.loc[:,simulation.features].sum(axis=1)

In [17]:
#df_2 = simulation.fourier_engineered_dataframe[simulation.fourier_engineered_dataframe.phase==2]
#df_2.describe()

In [18]:
#df_1 = simulation.fourier_engineered_dataframe[simulation.fourier_engineered_dataframe.phase==1]
#df_1.describe()

In [19]:
#df_0 = simulation.fourier_engineered_dataframe[simulation.fourier_engineered_dataframe.phase==0]
#df_0.describe()

In [20]:
#plt.figure(figsize=(10,10))
#plt.scatter(df_1.fourier_feat0,df_1.fourier_feat2)
#plt.scatter(df_0.fourier_feat0,df_0.fourier_feat2,alpha=0.4)

In [21]:
#plt.scatter(df_0.fourier_feat0,df_0.fourier_feat2,alpha=0.4)

In [22]:
#plt.scatter(df_2.fourier_feat0,df_2.fourier_feat2)

In [23]:
#df_1.hist(figsize=(15,15))

In [24]:
#df_0.hist(figsize=(15,15))

#### Checking initialization

In [25]:
%%time
n_features = simulation.n_features
n_hamiltonians = simulation.n_hamiltonians
n_ts = simulation.n_ts

print("n_features: ", n_features)
print("n_hamiltonians: ", n_hamiltonians)
print("n_ts: ", n_ts)

n_features:  100
n_hamiltonians:  6561
n_ts:  2
CPU times: user 153 µs, sys: 20 µs, total: 173 µs
Wall time: 97 µs


In [26]:
%%time
n_total = len(simulation.dataframe)
n_train = len(simulation.dataframe[simulation.dataframe.type_of == "train"])
n_val = len(simulation.dataframe[simulation.dataframe.type_of == "val"])
n_test = len(simulation.dataframe[simulation.dataframe.type_of == "test"])
n_train_hamiltonians = len(simulation.train_ids)
n_val_hamiltonians = len(simulation.val_ids)
n_test_hamiltonians = len(simulation.test_ids)
n_total_hamiltonians = n_train_hamiltonians + n_val_hamiltonians + n_test_hamiltonians

print("% train: ", n_train/n_total)
print("% val: ",  n_val/n_total)
print("% test: ", n_test/n_total)
print("% train + val + test: ", (n_train+n_val+n_test)/n_total)
print("\n")
print("number of train hamiltonians: ", n_train_hamiltonians)
print("number of val hamiltonians: ", n_val_hamiltonians)
print("number of test hamiltonians: ", n_test_hamiltonians)
print("total number of hamiltonians: ", n_total_hamiltonians)
print("\n")
print("train ids: ", simulation.train_ids)
print("val ids: ", simulation.val_ids)
print("test ids: ", simulation.test_ids)

% train:  0.846822130772748
% val:  0.0
% test:  0.15317786922725193
% train + val + test:  1.0


number of train hamiltonians:  5556
number of val hamiltonians:  0
number of test hamiltonians:  1005
total number of hamiltonians:  6561


train ids:  [9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201

## Running a simulation

In [27]:

%%time
simulation.run_simulation(n_experiments, start_n, fit_params,shuffle_rows, pred_params, random_features, \
                          store_in_lists, save_eigenvector, save_hamiltonian, save_accuracy,\
                          save_models)
if simulation.fourier_mode is not None:
    simulation.fourier_dataframe.head(10)

running experiments:   0%|          | 0/100 [00:00<?, ?it/s]

Simulation mode:  dst


running experiments:   1%|          | 1/100 [00:01<02:55,  1.77s/it]

running experiments:   2%|▏         | 2/100 [00:03<02:55,  1.79s/it]

running experiments:   3%|▎         | 3/100 [00:05<02:54,  1.80s/it]

...

running experiments:  98%|█████████▊| 98/100 [02:58<00:03,  1.82s/it]

running experiments:  99%|█████████▉| 99/100 [03:00<00:01,  1.79s/it]

running experiments: 100%|██████████| 100/100 [03:02<00:00,  1.83s/it]

running experiments: 100%|██████████| 100/100 [03:02<00:00,  1.82s/it]

CPU times: user 3min 1s, sys: 676 ms, total: 3min 2s
Wall time: 3min 2s


In [28]:
simulation.dataframe.head(10)

,id,path,t1,t2,winding,phase,pred_phase,type_of,feat0,feat1,...,feat90,feat91,feat92,feat93,feat94,feat95,feat96,feat97,feat98,feat99
0,0,H_-2_-2_0.509296.dat,-2.0,-2.0,0.509296,999,666,test,0.100000,0.100000,...,0.100000,0.100000,0.100000,0.100000,0.100000,0.100000,0.100000,0.100000,0.100000,0.100000
1,0,H_-2_-2_0.509296.dat,-2.0,-2.0,0.509296,999,666,test,0.141421,0.141142,...,0.114412,0.108967,0.123928,0.119406,0.131490,0.127962,0.136978,0.134500,0.140306,0.138916
2,0,H_-2_-2_0.509296.dat,-2.0,-2.0,0.509296,999,666,test,0.141421,0.140306,...,0.043702,0.026500,0.075777,0.060214,0.103092,0.090145,0.123928,0.114412,0.136978,0.131490
3,0,H_-2_-2_0.509296.dat,-2.0,-2.0,0.509296,999,666,test,-0.100000,0.100000,...,-0.100000,0.100000,-0.100000,0.100000,-0.100000,0.100000,-0.100000,0.100000,-0.100000,0.100000
4,0,H_-2_-2_0.509296.dat,-2.0,-2.0,0.509296,999,666,test,0.000509,-0.008372,...,-0.082713,-0.089753,-0.067684,-0.075347,-0.051587,-0.059753,-0.034677,-0.043217,-0.017220,-0.026000
5,0,H_-2_-2_0.509296.dat,-2.0,-2.0,0.509296,999,666,test,0.001204,-0.016530,...,-0.134123,-0.138686,-0.118757,-0.127445,-0.095929,-0.108196,-0.067073,-0.082149,-0.034003,-0.050940
6,0,H_-2_-2_0.509296.dat,-2.0,-2.0,0.509296,999,666,test,-0.141421,-0.138980,...,0.043375,0.067830,-0.009222,0.017385,-0.060524,-0.035502,-0.103326,-0.083403,-0.131616,-0.119589
7,0,H_-2_-2_0.509296.dat,-2.0,-2.0,0.509296,999,666,test,-0.000826,0.025688,...,0.134753,0.124324,0.141088,0.140407,0.127608,0.136771,0.096206,0.113925,0.051292,0.075079
8,0,H_-2_-2_0.509296.dat,-2.0,-2.0,0.509296,999,666,test,0.141417,0.136684,...,-0.115093,-0.131915,-0.061266,-0.091040,0.007717,-0.027643,0.074791,0.042592,0.123363,0.102291
9,0,H_-2_-2_0.509296.dat,-2.0,-2.0,0.509296,999,666,test,0.001302,0.036430,...,0.082068,0.050848,0.127402,0.108132,0.141218,0.138667,0.120099,0.134896,0.069268,0.097755


#### Viewing a random sample

In [29]:
%%time
if simulation.fourier_mode is not None:
    simulation.fourier_dataframe.sample(frac=0.1, replace=False)

CPU times: user 17.2 ms, sys: 0 ns, total: 17.2 ms
Wall time: 17 ms


#### Checking train/val/test splits again

In [30]:
%%time
n_total = len(simulation.dataframe)
n_train = len(simulation.dataframe[simulation.dataframe.type_of == "train"])
n_val = len(simulation.dataframe[simulation.dataframe.type_of == "val"])
n_test = len(simulation.dataframe[simulation.dataframe.type_of == "test"])
n_train_hamiltonians = len(simulation.train_ids)
n_val_hamiltonians = len(simulation.val_ids)
n_test_hamiltonians = len(simulation.test_ids)
n_total_hamiltonians = n_train_hamiltonians + n_val_hamiltonians + n_test_hamiltonians

print("% train: ", n_train/n_total)
print("% val: ",  n_val/n_total)
print("% test: ", n_test/n_total)
print("% train + val + test: ", (n_train+n_val+n_test)/n_total)
print("\n")
print("number of train hamiltonians: ", n_train_hamiltonians)
print("number of val hamiltonians: ", n_val_hamiltonians)
print("number of test hamiltonians: ", n_test_hamiltonians)
print("total number of hamiltonians: ", n_total_hamiltonians)
print("\n")
print("train ids: ", simulation.train_ids)
print("val ids: ", simulation.val_ids)
print("test ids: ", simulation.test_ids)

% train:  0.08474317939338516
% val:  0.7620789513793629
% test:  0.15317786922725193
% train + val + test:  1.0


number of train hamiltonians:  556
number of val hamiltonians:  5000
number of test hamiltonians:  1005
total number of hamiltonians:  6561


train ids:  [4402, 1513, 3136, 1910, 5224, 3876, 2678, 1339, 2660, 5114, 1381, 3482, 3448, 3387, 4600, 1122, 1817, 1676, 3743, 152, 3589, 3598, 3857, 1673, 4550, 6543, 4984, 3239, 3415, 5754, 3365, 3015, 3106, 5211, 3594, 2414, 1652, 6017, 2468, 4218, 5011, 56, 4355, 4696, 4908, 4404, 1465, 101, 3011, 3611, 1830, 2517, 5934, 1445, 1629, 2381, 5621, 2487, 2382, 1352, 2348, 1995, 4131, 5204, 2201, 3754, 5066, 1074, 1732, 2655, 5456, 355, 3001, 5438, 2785, 6365, 4422, 4988, 3013, 1285, 3434, 30, 4003, 6031, 5271, 5525, 1540, 2885, 3306, 1443, 2636, 4492, 937, 4224, 5038, 2337, 2986, 1376, 3016, 556, 5558, 3457, 2576, 6004, 5873, 6034, 4454, 3621, 3476, 3996, 1632, 4108, 2735, 2526, 109, 2180, 5212, 2508, 2505, 4646, 4859, 362, 1851, 256

#### Checking summaries

In [31]:
%%time
ham_summary = simulation.hamiltonian_summary
print("length of ham_summary: ", len(ham_summary))
ham_summary

length of ham_summary:  6561
CPU times: user 52 µs, sys: 1 µs, total: 53 µs
Wall time: 54.6 µs


,t1,t2,type_of,0,1,phase,pred_phase
id,,,,,,,
0,-2.0,-2.00,test,0.73,0.27,999,0
1,-2.0,-1.95,test,0.73,0.27,0,0
2,-2.0,-1.90,test,0.70,0.30,0,0
3,-2.0,-1.85,test,0.78,0.22,0,0
4,-2.0,-1.80,test,0.74,0.26,0,0
...,...,...,...,...,...,...,...
6556,2.0,1.80,test,0.72,0.28,0,0
6557,2.0,1.85,test,0.76,0.24,0,0
6558,2.0,1.90,test,0.75,0.25,0,0


In [32]:
ham_summary.describe()

,t1,t2,0,1,phase,pred_phase
count,6561.000000,6.561000e+03,6561.000000,6561.000000,6561.000000,6561.000000
mean,0.000000,-4.331917e-18,0.707564,0.292436,25.002134,0.164457
std,1.169134,1.169134e+00,0.195065,0.195065,154.495657,0.370718
min,-2.000000,-2.000000e+00,0.370000,0.000000,0.000000,0.000000
25%,-1.000000,-1.000000e+00,0.520000,0.100000,0.000000,0.000000
50%,0.000000,0.000000e+00,0.700000,0.300000,1.000000,0.000000
75%,1.000000,1.000000e+00,0.900000,0.480000,1.000000,0.000000
max,2.000000,2.000000e+00,1.000000,0.630000,999.000000,1.000000


In [33]:
eigen_summary = simulation.eigenvector_summary
print("length of ham_summary: ", len(eigen_summary))
eigen_summary

length of ham_summary:  656100


,id,phase,pred_phase,type_of
0,0,999,0,test
1,0,999,0,test
2,0,999,0,test
3,0,999,0,test
4,0,999,0,test
...,...,...,...,...
656095,6560,999,1,test
656096,6560,999,0,test
656097,6560,999,0,test
656098,6560,999,1,test


#### Checking accuracies

In [34]:
simulation.accuracy

{'eigenvector_train': 0.8157374100719424,
 'eigenvector_val': 0.695974,
 'eigenvector_test': 0.5307938388625593,
 'hamiltonian_train': 0.9928057553956835,
 'hamiltonian_val': 0.6698,
 'hamiltonian_test': 0.44786729857819907}

#### Checking data stored in  memory

In [35]:
ham_summary_list = simulation.hamiltonian_summary_list
ham_summary_list

[]

In [36]:
eigen_summary_list = simulation.eigenvector_summary_list
eigen_summary_list

[]

In [37]:
accuracy_list = simulation.accuracy_list
accuracy_list

{'eigenvector_train': [],
 'eigenvector_val': [],
 'eigenvector_test': [],
 'hamiltonian_train': [],
 'hamiltonian_val': [],
 'hamiltonian_test': []}

In [38]:
#simulation.fourier_matrix

In [39]:
#simulation.fourier_matrix.shape

In [40]:
#simulation.fourier_matrix[:,0]

In [41]:
#simulation.fourier_matrix[:,1]

In [42]:
#np.exp(-1j*2*np.pi*3/100)

In [43]:
#for i in range(1,50):
#    print("\n")
#    print(i)
#    print(np.sum((np.real(simulation.fourier_matrix[:,i]) - np.real(simulation.fourier_matrix[:,-i]))**2))
#    print(np.sum((np.imag(simulation.fourier_matrix[:,i])+ np.imag(simulation.fourier_matrix[:,-i]))**2))

In [44]:
#print(simulation.fourier_matrix[:,0])

In [45]:
#print(simulation.fourier_matrix[:,50])